## Detect hand-to-mouth eating motion using accelerometer and gyroscope data

### Model Training with LSTM

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

from google.colab import drive
drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)

%cd /content/drive/MyDrive/AI-Hand_to_Mouth_motion_detection

# Load your dataset
data = pd.read_csv('merged_sensor_data.csv')

# Assuming the dataset has columns: ['accel_z', 'accel_y', 'accel_x', 'gyro_z', 'gyro_y', 'gyro_x', 'label']
features = data[['accel_z', 'accel_y', 'accel_x', 'gyro_z', 'gyro_y', 'gyro_x']].values
labels = data['label'].values

# Encode the labels using LabelEncoder
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Normalize the data
scaler = StandardScaler()
features = scaler.fit_transform(features)

# Split data into sequences
def create_sequences(data, labels, time_steps=50):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i+time_steps])
        y.append(labels[i+time_steps])
    return np.array(X), np.array(y)

time_steps = 100
X, y = create_sequences(features, labels, time_steps)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert labels to numeric values
y_train = y_train.astype(int)
y_test = y_test.astype(int)

# Define the LSTM model
model = Sequential()
# model.add(LSTM(50, activation='relu', input_shape=(time_steps, X.shape[2])))
# model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(LSTM(50, activation='relu', input_shape=(time_steps, X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Compile model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

# Save the model
model.save('hand_to_mouth_model.h5')

Mounted at /content/drive
Mounted at /content/drive
/content/drive/MyDrive/AI-Hand_to_Mouth_motion_detection


Epoch 1/50
58/58 [==============================] - 15s 136ms/step - loss: 0.4011 - accuracy: 0.8081 - val_loss: 0.2000 - val_accuracy: 0.9369
Epoch 2/50
58/58 [==============================] - 8s 145ms/step - loss: 0.2216 - accuracy: 0.9246 - val_loss: 0.1387 - val_accuracy: 0.9554
Epoch 3/50
58/58 [==============================] - 7s 114ms/step - loss: 0.1388 - accuracy: 0.9510 - val_loss: 0.0951 - val_accuracy: 0.9717
Epoch 4/50
58/58 [==============================] - 9s 149ms/step - loss: 0.1182 - accuracy: 0.9611 - val_loss: 0.1158 - val_accuracy: 0.9521
Epoch 5/50
58/58 [==============================] - 7s 113ms/step - loss: 0.1403 - accuracy: 0.9363 - val_loss: 0.1005 - val_accuracy: 0.9674
Epoch 6/50
58/58 [==============================] - 8s 147ms/step - loss: 0.1086 - accuracy: 0.9494 - val_loss: 0.0879 - val_accuracy: 0.9674
Epoch 7/50
58/58 [==============================] - 7s 113ms/step - loss: 0.0986 - accuracy: 0.9545 - val_loss: 0.0758 - val_accuracy: 0.9717
Epoch

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Trained Model has high Test Accuracy: 98.87%

### Deployment
Once the model is trained and saved, load it for predictions on new data:

In [5]:
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('hand_to_mouth_model.h5')

# Function to predict on new data segments
def predict_hand_to_mouth(segments):
    predictions = model.predict(segments)
    return predictions > 0.5  # you may adjust the threshold based on your evaluation

# Split data into sequences
def create_new_sequences(data, time_steps=50):
    X = []
    for i in range(len(data) - time_steps):
        X.append(data[i:i+time_steps])
    return np.array(X)

%cd /content/drive/MyDrive/AI-Hand_to_Mouth_motion_detection

# Load your dataset
new_data = pd.read_csv('new_sensor_data.csv')

# Assuming the dataset has columns: ['accel_z', 'accel_y', 'accel_x', 'gyro_z', 'gyro_y', 'gyro_x']
new_features = new_data[['accel_z', 'accel_y', 'accel_x', 'gyro_z', 'gyro_y', 'gyro_x']].values

# Normalize the data
scaler = StandardScaler()
new_features = scaler.fit_transform(new_features)
time_steps = 100

# Preprocess new data
new_data= create_new_sequences(new_features, time_steps)

# new_segments = preprocess_new_data(new_df)

# Predict using the model
results = predict_hand_to_mouth(new_data)

# Post-process predictions
# Here, results is a list of boolean values indicating hand-to-mouth eating detection for each segment
for i, res in enumerate(results):
    if res:
        print(f"Segment {i}: Hand-to-mouth eating detected")
    else:
        print(f"Segment {i}: No hand-to-mouth eating detected")


/content/drive/MyDrive/AI-Hand_to_Mouth_motion_detection
94/94 [==============================] - 2s 19ms/step
Segment 0: No hand-to-mouth eating detected
Segment 1: No hand-to-mouth eating detected
Segment 2: No hand-to-mouth eating detected
Segment 3: No hand-to-mouth eating detected
Segment 4: No hand-to-mouth eating detected
Segment 5: No hand-to-mouth eating detected
Segment 6: No hand-to-mouth eating detected
Segment 7: No hand-to-mouth eating detected
Segment 8: No hand-to-mouth eating detected
Segment 9: No hand-to-mouth eating detected
Segment 10: No hand-to-mouth eating detected
Segment 11: No hand-to-mouth eating detected
Segment 12: No hand-to-mouth eating detected
Segment 13: No hand-to-mouth eating detected
Segment 14: No hand-to-mouth eating detected
Segment 15: No hand-to-mouth eating detected
Segment 16: No hand-to-mouth eating detected
Segment 17: No hand-to-mouth eating detected
Segment 18: No hand-to-mouth eating detected
Segment 19: No hand-to-mouth eating detected